In [27]:
import tkinter as tk
from tkinter import messagebox

class Kitap:
    def __init__(self, kitap_id, ad, yazar):
        self.kitap_id = kitap_id
        self.ad = ad
        self.yazar = yazar
        self.durum = "Rafta"  
    
    def durum_guncelle(self, durum):
        self.durum = durum

    def odunc_durumu(self):
        return self.durum == "Ödünçte"

    def to_dict(self):
        return {
            "kitap_id": self.kitap_id,
            "ad": self.ad,
            "yazar": self.yazar,
            "durum": self.durum
        }

class Uye:
    def __init__(self, uye_id, ad):
        self.uye_id = uye_id
        self.ad = ad

    def to_dict(self):
        return {
            "uye_id": self.uye_id,
            "ad": self.ad
        }

class Odunc:
    def __init__(self, kitap, uye):
        self.kitap = kitap
        self.uye = uye
        self.alim_tarihi = None
        self.iade_tarihi = None
    
    def odunc_al(self, alim_tarihi):
        self.alim_tarihi = alim_tarihi
        self.kitap.durum_guncelle("Ödünçte")
    
    def iade_et(self, iade_tarihi):
        self.iade_tarihi = iade_tarihi
        self.kitap.durum_guncelle("Rafta")
    
    def odunc_bilgisi(self):
        return f"{self.kitap.ad} kitabı, {self.uye.ad}'a ödünç verildi."


class KutuphaneSistemi:
    def __init__(self):
        self.kitaplar = [
            Kitap(1, "Dönüşüm", "Franz Kafka"),
            Kitap(2, "Beyaz Geceler", "Fyodor Dostoevsky"),
            Kitap(3, "Yabancı", "Albert Camus"),
            Kitap(4, "Ölü Canlar", "Nikolai Gogol"),
            Kitap(5, "Sefiller", "Victor Hugo")
        ]
        self.uyeler = []
        self.odunc_listesi = []

    def uye_ekle(self, uye):
        self.uyeler.append(uye)
    
    def kitap_bul(self, kitap_id):
        for kitap in self.kitaplar:
            if kitap.kitap_id == kitap_id:
                return kitap
        return None

class KutuphaneArayuzu:
    def __init__(self, kutuphane_sistemi):
        self.kutuphane_sistemi = kutuphane_sistemi
        self.root = tk.Tk()
        self.root.title("Kütüphane Sistemi")

        # Kullanıcı İsim Girişi
        self.kullanici_isim_cerceve = tk.LabelFrame(self.root, text="Kullanıcı İsmi")
        self.kullanici_isim_cerceve.pack(fill="both", expand="yes", padx=20, pady=10)
        tk.Label(self.kullanici_isim_cerceve, text="İsim:").grid(row=0, column=0)
        self.isim_entry = tk.Entry(self.kullanici_isim_cerceve)
        self.isim_entry.grid(row=0, column=1)
        self.isim_onay_button = tk.Button(self.kullanici_isim_cerceve, text="Onayla", command=self.kitap_secim)
        self.isim_onay_button.grid(row=1, columnspan=2)

        self.kitap_secim_cerceve = tk.LabelFrame(self.root, text="Kitap Seçimi")
        self.kitap_secim_cerceve.pack(fill="both", expand="yes", padx=20, pady=10)
        self.kitaplar_listesi = tk.Listbox(self.kitap_secim_cerceve)
        self.kitaplar_listesi.grid(row=0, column=0)

        for kitap in self.kutuphane_sistemi.kitaplar:
            self.kitaplar_listesi.insert(tk.END, f"{kitap.kitap_id}. {kitap.ad} - {kitap.yazar}")

        self.kitap_secim_button = tk.Button(self.kitap_secim_cerceve, text="Seç", command=self.kitap_odunc_al)
        self.kitap_secim_button.grid(row=1, column=0, pady=10)

        self.root.mainloop()

    def kitap_secim(self):
        kullanici_isim = self.isim_entry.get()
        self.kullanici = Uye(1, kullanici_isim)  # Geçici olarak sabit bir kullanıcı oluşturduk

    def kitap_odunc_al(self):
        secilen_index = self.kitaplar_listesi.curselection()
        if not secilen_index:
            messagebox.showerror("Hata", "Lütfen bir kitap seçin.")
            return

        secilen_index = secilen_index[0]
        secilen_kitap_id = secilen_index + 1  # 0'dan başladığı için +1 ekliyoruz

        kitap = self.kutuphane_sistemi.kitap_bul(secilen_kitap_id)
        if kitap:
            odunc = Odunc(kitap, self.kullanici)
            self.kutuphane_sistemi.odunc_listesi.append(odunc)
            messagebox.showinfo("Başarılı", f"{kitap.ad} kitabı {self.kullanici.ad} tarafından ödünç alındı.")
            self.kitaplar_listesi.delete(secilen_index)  # Ödünç alınan kitabı listeden kaldır
            iptal_button = tk.Button(self.kitap_secim_cerceve, text="İptal Et", command=lambda: self.kitap_iptal_et(kitap, secilen_index))
            iptal_button.grid(row=secilen_index+2, column=0)  # İptal et butonunu ekle
        else:
            messagebox.showerror("Hata", "Kitap bulunamadı.")

    def kitap_iptal_et(self, kitap, secilen_index):
        self.kutuphane_sistemi.odunc_listesi = [x for x in self.kutuphane_sistemi.odunc_listesi if x.kitap.kitap_id != kitap.kitap_id]
        self.kitaplar_listesi.insert(secilen_index, f"{kitap.kitap_id}. {kitap.ad} - {kitap.yazar}")
        messagebox.showinfo("Başarılı", f"{kitap.ad} kitabı iptal edildi.")

# Kullanım
if __name__ == "__main__":
    kutuphane_sistemi = KutuphaneSistemi()
    arayuz = KutuphaneArayuzu(kutuphane_sistemi)
